INSTALAÇÃO DE BIBLIOTECAS NECESSÁRIAS


In [16]:
!pip install xlwt
!pip install basedosdados
!pip install pandas
!pip install tabulate
!pip install os
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## **Preços de Combustíveis - Dados históricos**
#### **Organização**:    Agência Nacional do Petróleo, Gás Natural e Biocombustíveis (ANP)
##### **Cobertura temporal** : 2020 - 2021 (Mês de janeiro)

In [1]:
import basedosdados as bd
import pandas as pd

# Definir a query SQL para buscar apenas os dados de janeiro de 2021
sql_query = '''
SELECT *
FROM `basedosdados.br_anp_precos_combustiveis.microdados`
WHERE (EXTRACT(YEAR FROM data_coleta) = 2020 OR EXTRACT(YEAR FROM data_coleta) = 2021)
  AND EXTRACT(MONTH FROM data_coleta) = 1
'''

# Carregar apenas os dados filtrados para janeiro de 2021
df = bd.read_sql(sql_query, billing_project_id="leafy-ripsaw-382815")


Downloading: 100%|█████████████████████████████████████████████████████████| 325433/325433 [01:11<00:00, 4521.50rows/s]


#### Filtrando dataFrame com colunas adequadas ao projeto, removendo colunas que não irei utilizar.


In [2]:
dfcombustiveis = df.drop(['nome_estabelecimento','id_municipio','cep_revenda','cnpj_revenda','bairro_revenda','endereco_revenda','preco_compra'],axis=1)



#### convertendo todas as colunas para letras minúsculas, para facilitar a busca


In [3]:
dfcombustiveisLower = dfcombustiveis.applymap(lambda s: s.lower() if isinstance(s, str) else s)


## **Dados filtrados que estão dentro do dataframe escolhido**
#### Todos os dados abaixo podem ser pesquisados no DataFrame.

#### **Anos:**  2020,2021

#### **UF:**  rs, sp, rj, mg, am, ce, df, se, ro, pb, pe, ba, pa, al, ac, es, pr, rn, pi, to, ms, mt, go, rr, sc, ap, ma

#### **Revendedoras:**  branca, petrobras, distribuidora, s.a., raizen, ipiranga, nacional, gás, butano, copagaz, liquigás, ultragaz, supergasbras, energia, consigaz, rodoil, servgás, alesat, sabbá, "atem, s", equador, rzd, distribuidora, bahiana, setta, distribuidora, sp, dislub, amazongás, fogas, temape, atlântica, larco, minasgas, federal, simarelli, sul, combustíveis, petrox, distribuidora, stang, tobras, charrua, raizen, mime, small, taurus, royal, fic, idaza, uni, ciapetro, pelikano, potencial, ngc, distribuidora, total, brasil, tdc, distribuidora, maxsul, americanoil, rejaile, rio, branco, petrobahia, torrao, petroserra, fan, dibrape, masut, distribuidora, podium, hora, watt, ruff, c.j., walendowsky, montepetro, d`mais, gran, petro, supergasbras, dinamo, rm, petroleo, acol

#### **Produtos:**  diesel, etanol, gasolina, diesel, s10, gasolina, aditivada, gnv, glp

#### **Unidade de medida:**  R$/litro, R$/m3, R$/13kg

In [4]:
from tabulate import tabulate

# Define a lista de opções do menu
menu = [
    ["sigla: [1]", "Filtrar o dataframe pelas siglas - uf dos locais."],
    ["ano: [2]", "Filtrar o dataframe pelos anos de 2020 e 2021."],
    ["revenda: [3]", "Filtrar o dataframe pelas mais de 30 revendas."],
    ["produto: [4]", "Filtrar o dataframe pelos produtos gasolina, etanol, diesel"]
    
]

# Define a mensagem do cabeçalho da tabela
header = "Digite o número da opção que você quer filtrar:"

# Imprime a tabela na tela
print(tabulate(menu, headers=[header, "Descrição de cada opção"], tablefmt="grid"))


+---------------------------------------------------+-------------------------------------------------------------+
| Digite o número da opção que você quer filtrar:   | Descrição de cada opção                                     |
+===================================================+=============================================================+
| sigla: [1]                                        | Filtrar o dataframe pelas siglas - uf dos locais.           |
+---------------------------------------------------+-------------------------------------------------------------+
| ano: [2]                                          | Filtrar o dataframe pelos anos de 2020 e 2021.              |
+---------------------------------------------------+-------------------------------------------------------------+
| revenda: [3]                                      | Filtrar o dataframe pelas mais de 30 revendas.              |
+---------------------------------------------------+-------------------

##### **Anos:**  2020,2021

##### **UF:**  rs, sp, rj, mg, am, ce, df, se, ro, pb, pe, ba, pa, al, ac, es, pr, rn, pi, to, ms, mt, go, rr, sc, ap, ma

##### **Revendedoras:**  branca, petrobras, distribuidora, s.a., raizen, ipiranga, nacional, gás, butano, copagaz, liquigás, ultragaz, supergasbras, energia, consigaz, rodoil, servgás, alesat, sabbá, "atem, s", equador, rzd, distribuidora, bahiana, setta, distribuidora, sp, dislub, amazongás, fogas, temape, atlântica, larco, minasgas, federal, simarelli, sul, combustíveis, petrox, distribuidora, stang, tobras, charrua, raizen, mime, small, taurus, royal, fic, idaza, uni, ciapetro, pelikano, potencial, ngc, distribuidora, total, brasil, tdc, distribuidora, maxsul, americanoil, rejaile, rio, branco, petrobahia, torrao, petroserra, fan, dibrape, masut, distribuidora, podium, hora, watt, ruff, c.j., walendowsky, montepetro, d`mais, gran, petro, supergasbras, dinamo, rm, petroleo, acol

##### **Produtos:**  diesel, etanol, gasolina, diesel, s10, gasolina, aditivada, gnv, glp


In [5]:

while True:
  try:
    opcao = (input("Qual número da opção você deseja 1,2,3 ou 4 ? "))
    opcao = int(opcao)
    if opcao == 1:
      sigla = input("Digite o sigla UF: ").lower()
      dataframeFiltrado = dfcombustiveisLower[(dfcombustiveisLower['sigla_uf'] == sigla)]
      dataframeList = dataframeFiltrado.head().values.tolist()
      print(f"\n DataFrame - filtrado por: {sigla}\n")
      print(tabulate(dataframeList,headers=dfcombustiveis.columns, tablefmt='grid'))
    elif opcao == 2:
      ano = int(input("Digite o ano: "))
      dataframeFiltrado = dfcombustiveisLower[(dfcombustiveisLower['ano'] == ano)]
      dataframeList = dataframeFiltrado.head().values.tolist()
      print(f"\n DataFrame - filtrado por: {ano}\n")
      print(tabulate(dataframeList,headers=dfcombustiveis.columns, tablefmt='grid'))
    elif opcao == 3:
      revenda = str(input("Digite qual revendedora quer procurar: ")).lower()
      dataframeFiltrado = dfcombustiveisLower[(dfcombustiveisLower['bandeira_revenda'] == revenda)]
      dataframeList = dataframeFiltrado.head().values.tolist()
      print(f"\n DataFrame - filtrado por: {revenda}\n")
      print(tabulate(dataframeList,headers=dfcombustiveis.columns, tablefmt='grid'))
    elif opcao == 4:
      produto = input("Digite qual combustivel quer procurar: ").lower()
      dataframeFiltrado = dfcombustiveisLower[(dfcombustiveisLower['produto'] == produto)]
      dataframeList = dataframeFiltrado.head().values.tolist()
      print(f"\n DataFrame - filtrado por: {produto}\n")
      print(tabulate(dataframeList,headers=dfcombustiveis.columns, tablefmt='grid'))
    elif opcao == 0:
      print('Você saiu do programa, obrigado!!')
    break
  except ValueError:
    print("Valor inválido. Tente novamente.")
  

Qual número da opção você deseja 1,2,3 ou 4 ? A
Valor inválido. Tente novamente.
Qual número da opção você deseja 1,2,3 ou 4 ? E
Valor inválido. Tente novamente.
Qual número da opção você deseja 1,2,3 ou 4 ? ?
Valor inválido. Tente novamente.
Qual número da opção você deseja 1,2,3 ou 4 ? 1
Digite o sigla UF: SP

 DataFrame - filtrado por: sp

+-------+------------+------------------------------+---------------+------------+------------------+---------------+
|   ano | sigla_uf   | bandeira_revenda             | data_coleta   | produto    | unidade_medida   |   preco_venda |
+=======+============+==============================+===============+============+==================+===============+
|  2021 | sp         | petrobras distribuidora s.a. | 2021-01-01    | etanol     | r$/litro         |         2.999 |
+-------+------------+------------------------------+---------------+------------+------------------+---------------+
|  2021 | sp         | petrobras distribuidora s.a. | 2021-01-01 

#### DataFrame que foi filtrado:

In [6]:
dataframeFiltrado


,ano,sigla_uf,bandeira_revenda,data_coleta,produto,unidade_medida,preco_venda
1,2021,sp,petrobras distribuidora s.a.,2021-01-01,etanol,r$/litro,2.999
2,2021,sp,petrobras distribuidora s.a.,2021-01-01,gasolina,r$/litro,4.279
3,2021,sp,branca,2021-01-01,diesel s10,r$/litro,3.499
4,2021,sp,branca,2021-01-01,diesel s10,r$/litro,3.459
5,2021,sp,raizen,2021-01-01,diesel s10,r$/litro,3.799
...,...,...,...,...,...,...,...
325423,2021,sp,raizen,2021-01-31,gasolina aditivada,r$/litro,4.727
325424,2021,sp,ipiranga,2021-01-31,gasolina,r$/litro,4.599
325425,2021,sp,raizen,2021-01-31,diesel s10,r$/litro,3.829
325427,2021,sp,branca,2021-01-31,gasolina aditivada,r$/litro,4.681


In [21]:
import os
import xlwt
from pandas.io.formats.excel import ExcelFormatter
resposta = str(input("Você quer salvar o arquivo nos formatos xlsx e csv ? s/n ou qualquer teclas para finalizar...")).lower()
if resposta == 's':
  diretorio_destino = str(input("Digite o nome da pasta que deseja salvar o arquivo ou digite download para salvar no diretorio padrão.")).lower()
  if diretorio_destino == 'download':
    diretorio_destino = "C:\\Users\\cs376562\\Downloads"
  elif diretorio_destino != 'download':
    diretorio_destino = f"C:\\Users\\cs376562\\{diretorio_destino}"
  if not os.path.exists(diretorio_destino):
    os.makedirs(diretorio_destino)
  caminho_csv = os.path.join(diretorio_destino, 'dataframeCombustivel.csv')
  df.to_csv(caminho_csv, index=False)
  caminho_xlsx = os.path.join(diretorio_destino, 'dataframeCombustivel.xlsx')
  df.to_excel(caminho_xlsx, index=False)
  print("Arquivo salvo com sucesso.....")
else:
  print("Obrigado pela sua participação, agradeço que você não usou memoria da maquina para salvar o arquivo...... kkkkk")


Você quer salvar o arquivo nos formatos xlsx e csv ? s/n ou qualquer teclas para finalizar...s
Digite o nome da pasta que deseja salvar o arquivo ou digite download para salvar no diretorio padrão.combustiveis
Arquivo salvo com sucesso.....


In [7]:
# if resposta == 's':
#   dataframeFiltrado.to_csv('dataframeCombustivel.csv', index=False)
#   dataframeFiltrado.to_excel('dataframeCombustivel.xlsx', index=False)
#   print("Arquivo salvo com sucesso.....")
# else:
#   print("Obrigado pela sua participação, agradeço que você não usou memoria da maquina para salvar o arquivo...... kkkkk")



Arquivo salvo com sucesso.....


In [ ]:
# print('ano:', dfcombustiveisLower['ano'].unique())
# print('sigla_uf', dfcombustiveisLower['sigla_uf'].unique())
# print('bandeira_revenda', dfcombustiveisLower['bandeira_revenda'].unique())
# print('produto', dfcombustiveisLower['produto'].unique())
# print('unidade_medida', dfcombustiveisLower['unidade_medida'].unique())
# print('unidade_medida',dfcombustiveisLower['unidade_medida'].unique())